In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [29]:
df = pd.read_csv('/content/drive/MyDrive/슬기로운 건강생활/data_over_sampling.csv', encoding='cp949')
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,116,78,94,16.6,4,0
1,0,1,100,60,79,22.3,4,0
2,0,1,100,60,87,21.9,4,0
3,0,1,111,70,72,20.2,4,0
4,0,1,120,80,98,20.0,4,0
...,...,...,...,...,...,...,...,...
1481319,1,25,120,80,78,27.9,1,1
1481320,1,27,150,74,133,32.0,1,1
1481321,0,25,120,70,118,26.9,3,1
1481322,1,20,136,68,93,21.5,2,1


# ⭐데이터 나누는 것 해보기

In [16]:
df1 = df.sample(frac=1).reset_index(drop=True)
df1

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,1,9,109,68,89,20.8,4,0
1,0,15,126,74,141,23.7,2,1
2,1,3,90,60,85,21.5,4,0
3,1,14,138,80,100,23.5,4,0
4,0,3,134,85,94,23.5,4,0
...,...,...,...,...,...,...,...,...
1481319,0,18,126,81,92,26.7,1,1
1481320,1,17,120,80,132,25.2,1,1
1481321,1,21,119,70,92,32.0,2,1
1481322,0,15,127,71,79,21.7,4,0


In [6]:
# feature와 target 분리
yt=np.array(df['DIS'])
xt=df.drop(['DIS'], axis=1)
xt=np.array(xt)

# 데이터 정규화
scaler=MinMaxScaler()
xt=scaler.fit_transform(xt)

# train test 데이터 split
xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.2, random_state=42)

In [7]:
ytrain2 = ytrain
ytest2 = ytest

In [8]:
ytrain = np.array(xtrain['Dis'])
xtrain = xtrain.drop(['Dis'], axis=1)
xtrain=np.array(xtrain)

IndexError: ignored

In [ ]:
ytest = np.array(ytest['Dis'])
xtest = xtest.drop(['Dis'], axis=1)
xtest=np.array(xtest)

# ⭐ 랜덤포레스트

In [30]:
train=df.drop(['Dis'], axis=1)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481324 entries, 0 to 1481323
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   SEX     1481324 non-null  int64  
 1   BTH_G   1481324 non-null  int64  
 2   SBP     1481324 non-null  int64  
 3   DBP     1481324 non-null  int64  
 4   FBS     1481324 non-null  int64  
 5   BMI     1481324 non-null  float64
 6   DIS     1481324 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 79.1 MB


In [32]:
# 정답 4 모두 제거
train = train.loc[train['DIS']!=4]

In [33]:
train

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS
52,0,1,90,50,86,20.1,2
97,0,1,120,80,90,27.0,3
133,0,1,100,60,112,31.8,2
204,0,1,170,120,86,23.3,2
255,0,1,135,80,104,35.0,2
...,...,...,...,...,...,...,...
1481319,1,25,120,80,78,27.9,1
1481320,1,27,150,74,133,32.0,1
1481321,0,25,120,70,118,26.9,3
1481322,1,20,136,68,93,21.5,2


In [23]:
# feature와 target 분리
yt=np.array(train['DIS'])
xt=train.drop(['DIS'], axis=1)
xt=np.array(xt)

# 데이터 정규화
scaler=MinMaxScaler()
xt=scaler.fit_transform(xt)

# train test 데이터 split
xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.2, random_state=42)

In [24]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=17)
rf.fit(xtrain, ytrain)
forest_predictions = rf.predict(xtest)

accuracy_score(ytest, forest_predictions)
forest_predictions = rf.predict(xtest)

In [55]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [26]:
# 데이터 모두 사용

print(confusion_matrix(ytest, forest_predictions))

[[27275  2481   833]
 [ 1105 91141   601]
 [  992  1834 21871]]


In [27]:
print(classification_report(ytest, forest_predictions))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     30589
           2       0.95      0.98      0.97     92847
           3       0.94      0.89      0.91     24697

    accuracy                           0.95    148133
   macro avg       0.94      0.92      0.93    148133
weighted avg       0.95      0.95      0.95    148133



In [28]:
accuracy = accuracy_score(ytest, forest_predictions)
print('랜덤 포레스트 정확도: {:.4f}'.format(accuracy))

랜덤 포레스트 정확도: 0.9470


# train:test = 1:1

In [34]:
# feature와 target 분리
yt=np.array(train['DIS'])
xt=train.drop(['DIS'], axis=1)
xt=np.array(xt)

# 데이터 정규화
scaler=MinMaxScaler()
xt=scaler.fit_transform(xt)

# train test 데이터 split
xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.5, random_state=42)

In [35]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=17)
rf.fit(xtrain, ytrain)
forest_predictions = rf.predict(xtest)

accuracy_score(ytest, forest_predictions)
forest_predictions = rf.predict(xtest)

In [36]:
# 데이터 모두 사용

print(confusion_matrix(ytest, forest_predictions))

[[ 60933  11327   3683]
 [  4808 225397   2478]
 [  4824   8252  48629]]


In [37]:
print(classification_report(ytest, forest_predictions))

              precision    recall  f1-score   support

           1       0.86      0.80      0.83     75943
           2       0.92      0.97      0.94    232683
           3       0.89      0.79      0.83     61705

    accuracy                           0.90    370331
   macro avg       0.89      0.85      0.87    370331
weighted avg       0.90      0.90      0.90    370331



# train:test = 2:8

In [38]:
# feature와 target 분리
yt=np.array(train['DIS'])
xt=train.drop(['DIS'], axis=1)
xt=np.array(xt)

# 데이터 정규화
scaler=MinMaxScaler()
xt=scaler.fit_transform(xt)

# train test 데이터 split
xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.8, random_state=42)

In [39]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=17)
rf.fit(xtrain, ytrain)
forest_predictions = rf.predict(xtest)

accuracy_score(ytest, forest_predictions)
forest_predictions = rf.predict(xtest)

In [40]:
# 데이터 모두 사용

print(confusion_matrix(ytest, forest_predictions))

[[ 75510  35382  11405]
 [ 13357 351514   6830]
 [ 14001  25423  59108]]


In [41]:
print(classification_report(ytest, forest_predictions))

              precision    recall  f1-score   support

           1       0.73      0.62      0.67    122297
           2       0.85      0.95      0.90    371701
           3       0.76      0.60      0.67     98532

    accuracy                           0.82    592530
   macro avg       0.78      0.72      0.75    592530
weighted avg       0.81      0.82      0.81    592530



## 중요도 시각화
- [참고](https://moondol-ai.tistory.com/401)

In [44]:
!pip install eli5

     |████████████████████████████████| 216 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 60.0 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=f3e2f0f7b91ee68fdbd2cc1e3f7dfe530179758f4d17bd4a59d11bda376f4047
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [46]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=17).fit(xtrain, ytrain)

In [47]:
columns = xtrain.columns
coefficients = model.feature_importances_.reshape(xtrain.columns.shape[0], 1)
pd.concat([pd.DataFrame(columns, columns=['변수명']), pd.DataFrame(coefficients, columns=['특성 중요도'])], axis=1).sort_values(by='특성 중요도', ascending=False)

AttributeError: ignored

In [42]:
importance(rf)

NameError: ignored

# 오버샘플링(2)

In [50]:
df = pd.read_csv('/content/drive/MyDrive/슬기로운 건강생활/data(over_dis_vs_nor).csv', encoding='cp949')
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,90,50,86,20.1,2,1
1,0,1,120,80,90,27.0,3,1
2,0,1,100,60,112,31.8,2,1
3,0,1,170,120,86,23.3,2,1
4,0,1,135,80,104,35.0,2,1
...,...,...,...,...,...,...,...,...
1481319,1,21,130,90,185,26.3,3,1
1481320,1,18,140,100,120,28.8,2,1
1481321,0,12,100,60,101,23.0,3,1
1481322,1,20,134,72,168,23.1,3,1


In [51]:
train=df.drop(['Dis'], axis=1)

In [52]:
# 정답 4 모두 제거
train = train.loc[train['DIS']!=4]

In [53]:
# feature와 target 분리
yt=np.array(train['DIS'])
xt=train.drop(['DIS'], axis=1)
xt=np.array(xt)

# 데이터 정규화
scaler=MinMaxScaler()
xt=scaler.fit_transform(xt)

# train test 데이터 split
xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.2, random_state=42)

In [54]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=17)
rf.fit(xtrain, ytrain)
forest_predictions = rf.predict(xtest)

accuracy_score(ytest, forest_predictions)
forest_predictions = rf.predict(xtest)

In [56]:
# 데이터 모두 사용

print(confusion_matrix(ytest, forest_predictions))

[[47953  1007   486]
 [ 2794 44634  1636]
 [  331   386 48906]]


In [57]:
print(classification_report(ytest, forest_predictions))

              precision    recall  f1-score   support

           1       0.94      0.97      0.95     49446
           2       0.97      0.91      0.94     49064
           3       0.96      0.99      0.97     49623

    accuracy                           0.96    148133
   macro avg       0.96      0.96      0.95    148133
weighted avg       0.96      0.96      0.95    148133

